<a href="https://colab.research.google.com/github/aditya-malte/SemEval/blob/master/notebooks/SemEval_Loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install and Import Dependencies

In [1]:
!pip install emoji

In [0]:
#import fairseq
import pandas as pd

## Get URL function

In [0]:
def getUrl(text):
  processed_text = text[:]
  url = None
  processed_text, _, url = text.partition("…")
  if(url!=''):
    url = "".join(url.split())
    url = "https://t.co/" + url.rsplit('/', 1)[-1]
  else:
    url=None
  return (processed_text, url)

##Load Data

In [4]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
!ln -s "/gdrive/My Drive/SemEval_weights_data" "/content/"
drive_path = "/content/SemEval_weights_data/data/"
%cd {drive_path}

Mounted at /gdrive
ln: failed to create symbolic link '/content/SemEval_weights_data': File exists
/gdrive/My Drive/SemEval_weights_data/data


In [5]:
!ls

 cached_dev_xlm-roberta-base_128_sentimix     processed_val.csv
 cached_train_xlm-roberta-base_128_sentimix   processed_val.gsheet
 dev_3k_split_conll.txt			      train_14k_split_conll.txt
 dev.gsheet				      train.gsheet
 dev.tsv				     'train_scraped (1).gsheet'
 loaded_train.csv			      train_scraped.csv
 loaded_train.gsheet			      train_scraped.gsheet
 loaded_val.csv				      train.tsv
'processed_train (1).gsheet'		      valid_scraped.csv
 processed_train.csv			      val.tsv
 processed_train.gsheet


In [0]:
TRAIN_FILE = drive_path+"train_14k_split_conll.txt"
VAL_FILE = drive_path+"dev_3k_split_conll.txt" 

In [7]:
print(TRAIN_FILE)

/content/SemEval_weights_data/data/train_14k_split_conll.txt


In [0]:
raw_train_data = open(TRAIN_FILE)
train_data = pd.DataFrame(columns=['uid', 'text', 'lang_labels', 'url', 'sentiment'])
buffer = ""
lang_label_buffer = ""
sentiment = None
uid = None
for i, row in enumerate(raw_train_data):
  if(row == "\n"):
    continue
  try:
    row_split = row.split("\t")
    if(row_split[0]=="meta"):
      if(sentiment == None):
        sentiment = row_split[2]
        uid = row_split[1]
      else:
        train_data = train_data.append({"uid":uid.strip(), "text":buffer.strip(), "lang_labels":lang_label_buffer.strip(), "sentiment":sentiment.strip()}, ignore_index = True)
        buffer = ""
        lang_label_buffer = ""
        sentiment = row_split[2]
        uid = row_split[1]
    else:
      buffer = buffer + " " + row_split[0]
      lang_label_buffer = lang_label_buffer + " " +row_split[1].strip()
  except Exception as e:
    print("Exception:",e, ",Row",i,":",repr(row))
    raise
train_data = train_data.append({"uid":uid.strip(), "text":buffer.strip(), "lang_labels":lang_label_buffer.strip(), "sentiment":sentiment.strip()}, ignore_index = True)

In [0]:
def solveGaps(text): #convert @<space>xyz to @xyz and abc<space>_<space>xyz to abc_xyz
  return text.replace("@ ", "@").replace(' _ ', '_')

In [10]:
print("@ Nehr _ who @ TypoMantri @ anjanaomkashyap")
print(solveGaps("@ Nehr _ who @ TypoMantri @ anjanaomkashyap"))

@ Nehr _ who @ TypoMantri @ anjanaomkashyap
@Nehr_who @TypoMantri @anjanaomkashyap


In [11]:
train_data.head()

,uid,text,lang_labels,url,sentiment
0,4330,nen á vist bolest vztek smutek zmatek osam ě l...,Eng O Eng Eng Eng Eng Hin Hin O Eng Eng O Hin ...,NaN,neutral
1,41616,@ nehantics Haan yaar neha 😔😔 kab karega woh p...,O Hin Hin Hin Hin O Hin Hin Hin Hin EMT Hin Hi...,NaN,neutral
2,6648,@ RahulGandhi television media congress ke liy...,O Eng Eng Eng Eng Hin Hin Hin Hin O Hin Hin Hi...,NaN,negative
3,2512,@ AmitShah @ narendramodi All India me nrc lag...,O Hin O Hin Hin Hin Eng Hin Hin Hin Eng Hin Hi...,NaN,positive
4,610,@ Nehr _ who @ TypoMantri @ anjanaomkashyap Pa...,O Eng O Eng O Hin O Hin Hin Hin Hin O Eng Eng ...,NaN,neutral


In [0]:
train_data["text"] = train_data["text"].apply(lambda x: solveGaps(x))

## Save File

In [0]:
train_data[["text","url"]] = train_data.from_records(train_data["text"].apply(lambda x: getUrl(x)), columns=["text","url"])
train_data.to_csv(drive_path+"loaded_train.csv", index=False)

## Clear memory

In [0]:
train_data = None
raw_train_data = None

##Same steps for Validation data

In [0]:
raw_val_data = open(VAL_FILE)
val_data = pd.DataFrame(columns=['uid', 'text', 'lang_labels', 'url', 'sentiment'])
buffer = ""
lang_label_buffer = ""
sentiment = None
uid = None
for i, row in enumerate(raw_val_data):
  if(row == "\n"):
    continue
  try:
    row_split = row.split("\t")
    if(row_split[0]=="meta"):
      if(sentiment == None):
        sentiment = row_split[2]
        uid = row_split[1]
      else:
        val_data = val_data.append({"uid":uid.strip(), "text":buffer.strip(), "lang_labels":lang_label_buffer.strip(), "sentiment":sentiment.strip()}, ignore_index = True)
        buffer = ""
        lang_label_buffer = ""
        sentiment = row_split[2]
        uid = row_split[1]
    else:
      buffer = buffer + " " + row_split[0]
      lang_label_buffer = lang_label_buffer + " " +row_split[1].strip()
  except Exception as e:
    print("Exception:",e, ",Row",i,":",repr(row))
    raise
#add last row
val_data = val_data.append({"uid":uid.strip(), "text":buffer.strip(), "lang_labels":lang_label_buffer.strip(), "sentiment":sentiment.strip()}, ignore_index = True)

In [16]:
val_data.head()

,uid,text,lang_labels,url,sentiment
0,30258,@ prahladspatel modi mantrimandal may samil ho...,O Eng Eng Eng Eng Hin Hin Hin Hin Hin Hin,NaN,positive
1,16648,@ bkunalraj @ TajinderBagga @ NikhilJakhar14 @...,O Eng O Eng O Eng O Hin O Hin O Eng O Hin Hin ...,NaN,negative
2,28511,@ waglenikhil U saw caste and religion in them...,O Hin Hin Eng Eng Eng Eng Eng Eng O Eng Eng En...,NaN,negative
3,10466,@ DelhiPolice sir local police station pe comp...,O Hin Eng Eng Eng Hin Eng Eng Hin Hin Hin Hin ...,NaN,neutral
4,19266,Ve Maahi song from # Kesari is current favouri...,Hin Hin Eng Eng O Hin Eng Eng Eng O O Eng O En...,NaN,positive


## Save File

In [0]:
val_data[["text","url"]] = val_data.from_records(val_data["text"].apply(lambda x: getUrl(x)), columns=["text","url"])
val_data.to_csv(drive_path+"loaded_val.csv", index=False)